I should write stuff here

## Notes to self
- I removed clm, numclaims. Only focusing on claimcst0. Is there a way to train the model on clm, numclaims such that it somehow improves the accuracy???
- Alot of the claimcst0 have no values and only a few have non-zero value, is there something we can do here???

### Imports & Global Variables

In [51]:
import numpy as np
import pandas as pd
import matplotlib as plt
# place imports back here

In [52]:
import os, sys
PATH = %pwd
PATH = PATH.split(os.path.sep)
PATH = os.path.sep.join(PATH[:-2]) #removes last 2 directories, gets path to repository

### Preprocessing

In [53]:
# (1) Read both train/test + validation & preform equal preprocessing on it
df_t = pd.read_csv(os.path.join(PATH, 'dataset', 'InsNova_data_2023_train.csv'))
df_t['is_val'] = 0
df_v = pd.read_csv(os.path.join(PATH, 'dataset', 'InsNova_data_2023_vh.csv'))
df_v['is_val'] = 1
total_df = pd.concat([df_t,df_v], ignore_index=True)
del df_t, df_v


# Note
col_categorical = ['veh_body', 'gender', 'area', 'engine_type', 'veh_color', 
                'marital_status', 'time_of_week_driven', 'time_driven']
col_continuous = ['veh_value', 'exposure', 'credit_score']
col_discrete = ['veh_age', 'agecat', 'max_power', 'driving_history_score',
                'e_bill', 'trm_len', 'high_education_ind']


# (2) Feature Scaling
# Continuous/Discrete scaling
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()
total_df[col_continuous] = rs.fit_transform(total_df[col_continuous])
total_df[col_discrete] = rs.fit_transform(total_df[col_discrete])
# Categorical processing
total_df = pd.get_dummies(data=total_df,columns=col_categorical, dtype=float)


# (3) separate total_df
df_t = total_df[total_df['is_val'] == 0]
df_t = df_t.drop(columns=['is_val','clm','numclaims']) # !removing clm, numclaims
df_v = total_df[total_df['is_val'] == 1]
df_v = df_v.drop(columns='is_val')
del total_df

/Users/jb/anaconda3/envs/Kaggle-Travelers/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/jb/anaconda3/envs/Kaggle-Travelers/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/jb/anaconda3/envs/Kaggle-Travelers/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/jb/anaconda3/envs/Kaggle-Travelers/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_spar

### Data Separation & Packaging

In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from skorch import NeuralNetRegressor
from sklearn.metrics import mean_squared_error, r2_score
import random

### Apple Silicon
https://developer.apple.com/metal/pytorch/

(1)
curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh
sh Miniconda3-latest-MacOSX-arm64.s

(2)
conda install pytorch torchvision torchaudio -c pytorch-nightly
or
pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu


In [55]:
train_df, test_df = train_test_split(df_t, test_size=0.2, random_state=42)

X_train = train_df.drop('claimcst0', axis=1).values
y_train = train_df['claimcst0'].values
X_test = test_df.drop('claimcst0', axis=1).values
y_test = test_df['claimcst0'].values

# Ignore this code, apple cpu > apple gpu
device = "mps" if torch.backends.mps.is_available() else "cpu"
device = "cpu"
    
X_train = torch.FloatTensor(X_train).to(device)
y_train = torch.FloatTensor(y_train).view(-1, 1).to(device)  # Ensure the shape is (n_samples, 1)
X_test = torch.FloatTensor(X_test).to(device)
y_test = torch.FloatTensor(y_test).view(-1, 1).to(device)

Using device: mps


### Model Creation

In [56]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

### Model Training

In [57]:
# model initialization
HIDDEN = 64
LEARN_R = 0.001
EPOCHS = 1000
BATCH_S = 64

net = NeuralNetRegressor(
    module=SimpleNN(input_size=X_train.shape[1],
                    hidden_size=HIDDEN,
                    output_size=1
                    ).to(device),
    optimizer=optim.Adam,
    criterion=nn.MSELoss,
    max_epochs=EPOCHS,
    lr=LEARN_R,
    batch_size=BATCH_S,
    iterator_train__shuffle=True,
)

net.fit(X_train, y_train)
y_pred = net.predict(X_test)
torch.save(net, 'NN_Model.pth')

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R^2): {r2}")

random.seed(42)
num_samples_to_select = 5
random_indices = random.sample(range(len(X_test)), num_samples_to_select)
selected_data = X_test[random_indices]
true_values = y_test[random_indices]
predicted_values = net.predict(selected_data)
for i in range(num_samples_to_select):
    print(f"Sample {i+1} - Predicted: {predicted_values[i].item():.2f}, True Value: {true_values[i].item():.2f}")

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1  1540722.3232  1554911.7865  0.1358
      2  1535076.4128  1552842.4946  0.1226
      3  1536466.9335  1554613.9288  0.1462
      4  1537439.7240  1554785.9853  0.1947
      5  1536309.9673  1552409.6872  0.1782
      6  1536087.8166  1562017.0007  0.1338
      7  1536207.3826  1553672.5134  0.1277
      8  1536422.7917  1554236.7378  0.1343
      9  1535589.0283  1552835.6045  0.1431
     10  1534663.7188  1555986.1456  0.1382
     11  1536070.9548  1552775.4650  0.1309
     12  1534922.9987  1554829.8504  0.1388
     13  1535792.3264  1552325.8884  0.1447
     14  1534916.8438  1552209.8530  0.1340
     15  1535606.7244  1552171.8166  0.1333
     16  1535436.6309  1553654.6161  0.1423
     17  1535484.2496  1553269.6744  0.1404
     18  1534553.1485  1555664.7016  0.1294
     19  1535191.9690  1552218.1356  0.1314
     20  1534989.1162  1552469.1878  0.1400
     21  1535807.7673  1552151.0